In [ ]:
!pip install sentence-transformers faiss-cpu chromadb

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import uuid

ImportError: cannot import name 'TypeIs' from 'typing_extensions' (/opt/conda/lib/python3.11/site-packages/typing_extensions.py)

In [ ]:
# 1. Start Spark session
spark = SparkSession.builder.getOrCreate()

# 2. Read documents from HDFS (e.g., text or extracted PDF)
df = spark.read.text("hdfs://hadoop-namenode:9000/data/sample.txt")
df.show()

# 3. Convert to plain Python list
text_lines = df.select("value").rdd.flatMap(lambda x: x).collect()
full_text = "\n".join(text_lines)

# 4. Chunk text (simple fixed-size chunks)
def chunk_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

chunks = chunk_text(full_text)

# 5. Embed chunks
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)

# 6. Store in FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Save the index and chunks for later retrieval
faiss.write_index(index, "faiss_index.idx")

# Save chunk metadata
with open("chunk_lookup.txt", "w") as f:
    for i, chunk in enumerate(chunks):
        f.write(f"{i}\t{chunk[:100]}...\n")

print(f"✅ Stored {len(chunks)} chunks in FAISS index")